In [1]:
import sys
import os
import shutil
import glob

import GPRutils
import vK2KGPR
import plotGPR
import vonkarmanFT as vk

import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt

from astropy.modeling import models, fitting

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [2]:
gBand = sorted(np.array([
    474262,
    361580,
    474265,
    696547,
    484483,
    367484,
    369801,
    576863,
    791229,
    791593,
    791640,
    579816
]))
gBand

[361580,
 367484,
 369801,
 474262,
 474265,
 484483,
 576863,
 579816,
 696547,
 791229,
 791593,
 791640]

In [ ]:
for exp in gBand:
    expFile = os.path.join("/home/fortino/gBand_rerun", str(exp))
    try:
        os.mkdir(expFile)
    except FileExistsError:
        shutil.rmtree(expFile)
        os.mkdir(expFile)
        
    sys.stderr = open(os.path.join(expFile, "err.err"), "a+")
    sys.stdout = open(os.path.join(expFile, "out.out"), "a+")
    
    dataC = GPRutils.dataContainer()
    dataC.load(exp)
    
    GP = vK2KGPR.vonKarman2KernelGPR(dataC)
    GP.fitCorr()
    GP.fit(GP.opt_result[0])
    dataC.postFitCorr_sigmaClip(GP)
    GP.optimize()
    dataC.JackKnife(GP)
    dataC.saveFITS(expFile)